In [ ]:
%pip install --user numpy
%pip install --user matplotlib

In [3]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [392]:
class MultiLayerPerceptronClassifier:
    
    def __init__(self, hidden_layer, suppress_output = False):
        self.W = None
        self.b = None
        self.suppress_output = False
        self.hidden_layer = hidden_layer
        self.outputs = None
        self.A = None
        
    def fit(self, X, y):
        #Se hará la suposición que todas las filas de X tienen el mismo número de columnas.
        self.build_network(len(X[0]))
        
        index = math.floor(random.random()*len(X))
        
        y_pred = self.feed_forward(X[index])
        print("FF: %s " % y_pred)
        
        self.back_propagation(X[index], y[index], y_pred, learning_rate=0.01)

    
    #input_length esero entero positivo mayor a 1 que representa la cantidad de atributos de cada
    #dato de entrada (sin incluir el que se busca predecir).
    #hidden_dimensions es una lista que contiene las dimensiones de las capas escondidas de la red
    #si hidden_dimensions es [3,3,3] se crearán tres capas escondidas de 
    def build_network(self, input_length):
        
        dimensions = self.hidden_layer                   
        
        #W matriz de pesos. 3D, primera dimensión capa, Segunda dimensión neurona, tercera dimensión pesos. 
        #Entonces, W[0][0] contendría el arreglo con los pesos de la neurona 0 de la capa 0.
        W = []  
        for i in range(len(dimensions)): 
            W.append([])
        
        
        #b es un arreglo 2D que contiene el bias de cada neurona b[0][0] contiene el bias de la neurona 0 
        #de la capa 0
        b = []
        for i in range(len(dimensions)): 
            b.append([])
        
        
        #todas las neuronas en la primera capa deben tener input_lenght pesos. De ahí en adelante, 
        #cada neurona en la capa j tendrá len(W[j-1]) pesos.
        layer_width = dimensions[0]
        
        for i in range(layer_width):
            W[0].append([])
            W[0][i] = [random.uniform(-1,1) for j in range(input_length)]
            b[0].append(0)
            b[0][i] = random.uniform(-1,1)
            W[0][i] = np.array([W[0][i]])
            

            
        #Inicialización aleatoria de pesos y bias entre -1 y 1
        for i in range(1, len(dimensions)):
            layer_width = dimensions[i]
            for j in range(layer_width):
                #se inicializan n valores aleatorios entre -1 y 1 donde n = len(W[i-1])
                W[i].append([])
                W[i][j] = [random.uniform(-1,1) for j in range(len(W[i-1]))]
                b[i].append(0)
                #se crea un bias aleatorio para la neurona
                b[i][j] = random.uniform(-1,1) 
            
                W[i][j] = np.array([W[i][j]])
                
        
        #Ahora se insertan los pesos y bias al output unit
        W.append([])
        W[len(W) - 1].append([])
        b.append([])
        b[len(W) - 1].append(0)
        W[len(W) - 1][0] = [random.uniform(-1,1) for j in range(len(W[len(W)-2]))]
        W[len(W) - 1][0] = np.array([W[len(W) - 1][0]])
        b[len(W) - 1][0] = random.uniform(-1,1)
            
        self.W = W
        self.b = b
        
        if not self.suppress_output:
            print("Network Built")
            print("W")
            print(W)
            print("b")
            print(b)
            print("--------------------------------------------------------------------")
            
    #X es una entrada
    #W matriz de pesos. 3D, primera dimensión capa, Segunda dimensión neurona, tercera dimensión pesos. 
    #Entonces, W[0][0] contendría el arreglo con los pesos de la neurona 0 de la capa 0.
    #l será el número de capas.
    #b es un arreglo 2D que contiene el bias de cada neurona b[0][0] contiene el bias de la neurona 0 
    #de la capa 0.
    #Algoritmo se basa en el algoritmo 6.3 de GoodFellow et al 
    #y en el algoritmo de la página 217 de Grus en Data Science from Scratch
    def feed_forward(self, x):
        
        l = len(self.hidden_layer)
        
        #Salidas de cada neurona. Primera dimensión capa, segunda neuronas
        #outputs[0][0] representa la entrada 0 de X. 
        #outputs[1][0] representa la salida 0 de la primera capa escondida.
        outputs = [np.array([x])]
        A = []
        
        for layer in range(1,l):
            
            layer_out = []
            A_out = []
            for neuron in range(len(self.W[layer])):
                #a es el resultado de las salidas de la capa anterior por los pesos de la capa actual.
                
                X = outputs[layer-1].transpose()
                W = self.W[layer-1][neuron]
                
                a = np.dot(W,X)[0][0] + self.b[layer][neuron]
                layer_out.append(self.relu(a))
                A_out.append(a)
                
            outputs.append(np.array([layer_out]))
            A.append(np.array([A_out]))
            
        #Ahora se computa el output del output layer. 
        X = outputs[len(self.W)-2].transpose()
        W = self.W[len(self.W)-1][0]
        np.dot(W,X)[0][0]
        self.b[len(self.W)-1][0]
        a = np.dot(W,X)[0][0] + self.b[len(self.W)-1][0]
        outputs.append(self.sigmoid(a))
        A.append(a)
        
        if not self.suppress_output:
            print("----------------------Feed_forward outputs--------------------------")
            print(outputs)
            print("----------------------Feed_forward A's------------------------------")
            print(A)
            print("--------------------------------------------------------------------")
        
        
        #Retorna el sigmoide del último output. Esto representa la probabilidad de que 
        #x pertenezca a la clase 1.
        self.outputs = outputs
        self.A = A
        return outputs[len(outputs)-1]
    
    #Relu para hidden units
    def relu(self, z):
        return max(0,z)
    
    #Sigmoide para Output unit
    def sigmoid(self, x):
        if x < 0:
            return 1 - 1/(1+math.exp(x))
        else:
            return 1/(1+math.exp(-x))
    
    def relu_derivative(self, z): 
        if z > 0:
            return 1
        else:
            return 0
    
    
    def sigmoid_derivative(self, z):
        s = self.sigmoid(z)
        return s*(1-s)
    
    #Sean X e y los datos escogidos para realizar la estimación en línea del gradiente.
    def back_propagation(self, x, y, y_pred, learning_rate):
        #Activación traspuesta producto raro W transpuesta producto error del siguiente. 
        #Matriz bidimensional que contiene los errores de las capas escondidas. 
        
        #Los pesos del output se ajustan, usando descenso de gradiente, con base a e
        W = self.W
       
        #TODO, no estoy seguro si se usa este error o el error del perceptrón
        #e = y - y_pred
        #estimated_grad = np.multiply(e, x)
        #
        #W[len(W) - 1] = W[len(W) - 1] + np.multiply(learning_rate, estimated_grad)
        
        #Compute the gradient of the output layer
        #en linea, se utiliza únicamente la pareja (x,y) para estimar.
        g = y - y_pred
        delta_out = np.multiply(g, self.sigmoid(y_pred))
                
        delta = []
        #deltas de las capas escondidas + out layer
        for i in range(len(self.hidden_layer)+1):
            delta.append(np.array([]))
        
        delta[len(self.hidden_layer)] = delta_out
          
        #print(len(self.outputs))
        #print(len(self.hidden_layer))
        for i in range(len(self.hidden_layer)-1, -1, -1):
            #Calcular f' de las activaciones
            a = self.outputs[i][0]
            #f' de los outputs
            f_a_prime = [self.relu_derivative(i) for i in a]
            f_a_prime = np.array([f_a_prime]).transpose()
            
            
            print("f_a_prime")
            print(f_a_prime)
            
            g = np.multiply(g, f_a_prime)
        
            
            h = self.outputs[i-1]
            
            print("g")
            print(g)
            print("h")
            print(h)
            
            w_grad = np.dot(g, h)
            b_grad = g
            
            print("w_grad, b_grad")
            print(w_grad)
            print(b_grad)
            
            #g = (W_t^T)g
            print("W_k")
            print(W[i])

            g = np.dot(W[i].transpose(), g)
            print("new g")
            print(g)
            
        #for i in range(len(self.hidden_layer)):
        #    delta.append([])
        
        #Calcular error de la última capa escondida
        #(caso especial)
        
        
        self.W = W
        
        #Los pesos de acá multiplicado por el error de la siguiente capa
        #np.dot(W[i+1].transpose(), e)
        
        #np.multiply(outputs[i].transpose(), )
        
        return None

In [378]:
data_matrix = np.loadtxt(open("./msd_genre_dataset/fixed_ds.csv", "r"), delimiter=",", skiprows=0)



print("Filas de la matriz: " + str(len(data_matrix)))
print("Columnas de la matriz: " + str(len(data_matrix[0])))

y = data_matrix[:,len(data_matrix[0])-1]
X = np.delete(data_matrix, len(data_matrix[0])-1,1)

y = y.astype(int)


print(X)
print(y)


#Se intenta estandarizar X para lograr mejor desempeño. Sin embargo, no parece funcionar.
X = MinMaxScaler().fit_transform(X)

#Los datos del set de datos están agrupados por género. Es decir, primero están todas las filas que corresponden
#a 1 y después todas las que corresponden a -1. Se hace un shuffle para que, más tarde, en cross-validation
#no se creen unos modelos que predigan únicamente una clase.
np.random.shuffle(data_matrix)


print("X shape" + str(X.shape))
print("y shape" + str(y.shape))

Filas de la matriz: 8330
Columnas de la matriz: 31
[[-18.996       89.147        1.         ... 233.20616681 261.85070337
  240.83417734]
 [-19.347      125.825        4.         ... 227.2751235  261.64357048
  332.35653566]
 [ -9.472      121.707        4.         ... 549.49321044 481.14904868
  442.66313626]
 ...
 [ -9.494       88.976        4.         ... 519.52773394 538.89585608
  313.77593362]
 [ -7.617       67.929        3.         ... 591.85304812 598.05409088
  443.89380682]
 [-11.774       85.176        3.         ... 659.32142175 531.85019809
  607.21596134]]
[ 1  1  1 ... -1 -1 -1]
X shape(8330, 30)
y shape(8330,)


In [393]:
mlp = MultiLayerPerceptronClassifier(hidden_layer = [3, 3, 3], suppress_output = False)

mlp.fit(X, y)

Network Built
W
[[array([[ 0.33689729,  0.29313145, -0.90371925, -0.58911444,  0.29916219,
         0.91794687, -0.39709357,  0.87340645,  0.15178005,  0.7773885 ,
        -0.49769426,  0.19100682, -0.86014261, -0.40219076,  0.37868386,
        -0.09595167,  0.68328883,  0.53901887,  0.39866176,  0.35978874,
         0.83656233,  0.08068399, -0.69733708,  0.12641612,  0.56926875,
         0.74584889, -0.29744623,  0.31011283,  0.24959301, -0.55466537]]), array([[-0.29802593, -0.14709702, -0.96916042, -0.89725089, -0.00132237,
         0.14512391,  0.30797551, -0.14070085,  0.86141915, -0.31712035,
        -0.90893442, -0.60702624, -0.20808579,  0.07650591, -0.40341241,
         0.1802478 , -0.70916021,  0.41703984, -0.54069441, -0.44703756,
         0.01845403, -0.64091321,  0.12624951, -0.32155099,  0.75745946,
         0.46582408, -0.98438244,  0.74805914,  0.79402735,  0.18365923]]), array([[ 0.65026178,  0.58659524, -0.19044691,  0.86201524,  0.94326603,
         0.08993959, -0.868

AttributeError: 'list' object has no attribute 'transpose'